# Note:
* system prompt is not consistent on the effect, especially for the later rounds
     - maybe using system prompt everytime before a message?
* when conversation is initialized, it usually starts with greetings, makes behavior fixed in many ways
* 

Crowd influence (collectivism)

tipping points

In [1]:
import sys
sys.path.append('../src')
from conversation import run_single_simulation
from figure import draw_lineplot
import multiprocessing as mp
import pickle

In [3]:
key1 = "concise"
key2 = None
repeat = 10
result_dir = f"../results/{key1}/"

if __name__ == "__main__":
    # create folder
    import os
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    num_processes = min(mp.cpu_count(), 1)  # Use up to 10 CPU cores
    arg_list = [(key1, key2) for i in range(repeat)]
    with mp.Pool(processes=num_processes) as pool:
        results = pool.map(run_single_simulation, arg_list)  # Run 10 conversations in parallel

    # Unpacking results
    agent1_ratings, agent2_ratings, conversations = zip(*results)

    # Save results
    with open(f'{result_dir}r1_{key1}_simple.pkl', 'wb') as f:
        pickle.dump(list(agent1_ratings), f)

    with open(f'{result_dir}r2_{key1}_simple.pkl', 'wb') as f:
        pickle.dump(list(agent2_ratings), f)

    with open(f'{result_dir}c_{key1}_simple.pkl', 'wb') as f:
        pickle.dump(list(conversations), f)

    draw_lineplot(agent1_ratings, agent2_ratings, key1, result_dir)

[{'role': 'system', 'content': '# Background: This is a conversation between two people.\n # Persona: You are concise.\n # Task: Initiate the conversation.'}]
[{'role': 'system', 'content': '# Background: This is a conversation between two people.'}, {'role': 'user', 'content': "Hey! How's your day going?"}]
[{'role': 'system', 'content': '# Background: This is a conversation between two people.\n # Persona: You are concise.'}, {'role': 'assistant', 'content': "Hey! How's your day going?"}, {'role': 'user', 'content': "Hey! I'm doing well, thanks for asking. How about you? How's your day going?"}]
[{'role': 'system', 'content': '# Background: This is a conversation between two people.'}, {'role': 'user', 'content': "Hey! How's your day going?"}, {'role': 'assistant', 'content': "Hey! I'm doing well, thanks for asking. How about you? How's your day going?"}, {'role': 'user', 'content': "I'm doing well, thanks! Just here, ready to chat. What’s on your mind today?"}]
[{'role': 'system', '

Process SpawnPoolWorker-2:


KeyboardInterrupt: 

In [11]:
import pickle

with open(f"../results/{key1}/c_{key1}_simple.pkl", "rb") as f:
    c = pickle.load(f)

In [12]:
c[0]

[{'role': 'user',
  'content': "Hello there! How are you doing today? Is there anything specific on your mind or a particular topic you'd like to discuss? I'm here to chat!"},
 {'role': 'assistant',
  'content': "Hello! I'm doing well, thank you for asking. I'm here to help with any topic you're interested in—whether it's a question you have, something you want to learn about, or just casual conversation. What’s on your mind?"},
 {'role': 'user',
  'content': "I'm glad to hear you're doing well! There are so many fascinating topics we could dive into. How about we talk about something that piques your interest? It could be anything from technology advancements, books you've read recently, or even your favorite hobbies. What do you feel like discussing today?"},
 {'role': 'assistant',
  'content': 'I appreciate your enthusiasm! I’m always interested in technology advancements, especially in areas like artificial intelligence, renewable energy, or space exploration. Each of these fields 